# Social Media Advertisement Performance Analysis  

This notebook analyzes the performance of social media ads across multiple campaigns, platforms, and demographics.  
We focus on three main objectives:  

1. **Funnel Conversion Analysis** – Impression → Click → Purchase efficiency.  
2. **Audience Segmentation Insights** – Who engages and converts best (by gender, age, country).  
3. **Campaign ROI & Budget Efficiency** – Which campaigns deliver the most value for spend.  

---

## Data Dictionary

- **Users**: Detailed individual profiles (age, gender, country, location, interests). Used for demographic segmentation.  
- **Campaigns**: High-level campaign data (budget, duration, start/end date).
- **Ads**: Creative assets tied to campaigns, with targeting parameters (age, gender, interests, platform). Key for evaluating targeting effectiveness.  
- **Ad Events**: Transactional log of all user interactions (impression → click → purchase). Important for the conversion funnel.  

**Metrics**  
- **CTR (Click-Through Rate)**: percentage (%) of impressions that result in clicks → measures ad effectiveness.  
- **CVR (Conversion Rate)**: (%) of clicks that result in purchases → measures landing page persuasiveness.  


## Data Import & Initial Inspection
Importing the four core datasets and inspecting their structure, checking the columns, data types, etc.  

In [64]:
import pandas as pd

ads = pd.read_csv("../data/ads.csv")
ad_events = pd.read_csv("../data/ad_events.csv")
campaigns = pd.read_csv("../data/campaigns.csv")
users = pd.read_csv("../data/users.csv")

print(ads.tail(10))
print(ad_events.tail(10))
print(campaigns.tail(10))
print(users.tail(10))


     ad_id  campaign_id ad_platform   ad_type target_gender target_age_group  \
190    191           19    Facebook     Image           All            25-34   
191    192           47    Facebook  Carousel        Female            35-44   
192    193            5   Instagram  Carousel           All            25-34   
193    194           41    Facebook     Image           All            18-24   
194    195            9   Instagram  Carousel        Female            35-44   
195    196           12    Facebook   Stories          Male            35-44   
196    197            9    Facebook   Stories           All              All   
197    198           34    Facebook     Video           All            35-44   
198    199           15   Instagram     Video          Male            25-34   
199    200           44   Instagram  Carousel          Male            18-24   

        target_interests  
190            lifestyle  
191              fashion  
192              finance  
193        

## Data Cleaning  
To ensure the data is ready for analysis, steps taken are:  
- Convert all string columns to lowercase.  
- Remove whitespace.  
- Ensure proper datetime formatting for timestamps and dates.

In [65]:
def clean_columns(df, cols):
    for col in cols:
        df[col] = df[col].str.lower().str.strip()

# convert all string values to lowercase, erase any whitespaces
clean_columns(ads, ['ad_platform', 'ad_type', 'target_gender'])
clean_columns(ad_events, ['day_of_week', 'time_of_day', 'event_type'])
clean_columns(users, ['user_gender', 'country', 'location'])
campaigns['name'] = campaigns['name'].str.lower().str.strip()

ad_events['timestamp'] = pd.to_datetime(ad_events['timestamp'], errors='coerce')
print(ad_events['timestamp'].dtypes)  
print(ad_events['timestamp'].isnull().sum())

campaigns['start_date'] = pd.to_datetime(campaigns['start_date'], errors='coerce')
print(campaigns['start_date'].dtypes)
print(campaigns['start_date'].isnull().sum())

campaigns['end_date'] = pd.to_datetime(campaigns['end_date'], errors='coerce')
print(campaigns['end_date'].dtypes)
print(campaigns['end_date'].isnull().sum())



datetime64[ns]
0
datetime64[ns]
0
datetime64[ns]
0


Check if data is cleaned and standardized, check for any null values

In [66]:
print(ads.tail(10))
print(ad_events.tail(10))
print(campaigns.tail(10))
print(users.tail(10))

     ad_id  campaign_id ad_platform   ad_type target_gender target_age_group  \
190    191           19    facebook     image           all            25-34   
191    192           47    facebook  carousel        female            35-44   
192    193            5   instagram  carousel           all            25-34   
193    194           41    facebook     image           all            18-24   
194    195            9   instagram  carousel        female            35-44   
195    196           12    facebook   stories          male            35-44   
196    197            9    facebook   stories           all              All   
197    198           34    facebook     video           all            35-44   
198    199           15   instagram     video          male            25-34   
199    200           44   instagram  carousel          male            18-24   

        target_interests  
190            lifestyle  
191              fashion  
192              finance  
193        

In [67]:
ads.isnull().sum()

ad_id               0
campaign_id         0
ad_platform         0
ad_type             0
target_gender       0
target_age_group    0
target_interests    0
dtype: int64

In [68]:
ads.duplicated().sum()

np.int64(0)

In [69]:
ad_events.isnull().sum()

event_id       0
ad_id          0
user_id        0
timestamp      0
day_of_week    0
time_of_day    0
event_type     0
dtype: int64

In [70]:
ad_events.duplicated().sum()

np.int64(0)

In [71]:
campaigns.isnull().sum()

campaign_id      0
name             0
start_date       0
end_date         0
duration_days    0
total_budget     0
dtype: int64

In [72]:
campaigns.duplicated().sum()

np.int64(0)

In [73]:
users.isnull().sum()

user_id        0
user_gender    0
user_age       0
age_group      0
country        0
location       0
interests      0
dtype: int64

In [74]:
users.duplicated().sum()

np.int64(0)

In [75]:
ads.describe() 

,ad_id,campaign_id
count,200.000000,200.000000
mean,100.500000,25.125000
std,57.879185,13.713485
min,1.000000,1.000000
25%,50.750000,13.000000
50%,100.500000,25.000000
75%,150.250000,37.000000
max,200.000000,50.000000


In [76]:
ad_events.describe()

,event_id,ad_id,timestamp
count,400000.000000,400000.000000,400000
mean,200000.500000,100.535965,2025-06-22 01:09:12.365887744
min,1.000000,1.000000,2025-05-07 14:11:57
25%,100000.750000,51.000000,2025-05-30 07:55:34
50%,200000.500000,101.000000,2025-06-22 00:15:13.500000
75%,300000.250000,151.000000,2025-07-14 19:47:24.249999872
max,400000.000000,200.000000,2025-08-06 14:11:30
std,115470.198175,57.775915,NaN


In [77]:
campaigns.describe()

,campaign_id,start_date,end_date,duration_days,total_budget
count,50.00000,50,50,50.000000,50.000000
mean,25.50000,2025-04-30 02:52:48,2025-07-05 03:50:24,66.040000,50718.475600
min,1.00000,2025-02-13 00:00:00,2025-04-02 00:00:00,32.000000,7918.040000
25%,13.25000,2025-03-30 12:00:00,2025-06-01 18:00:00,52.500000,31105.435000
50%,25.50000,2025-04-23 12:00:00,2025-07-01 12:00:00,69.500000,48053.655000
75%,37.75000,2025-05-26 18:00:00,2025-08-04 12:00:00,81.750000,71600.592500
max,50.00000,2025-07-23 00:00:00,2025-10-12 00:00:00,90.000000,98904.660000
std,14.57738,NaN,NaN,16.380626,24576.018632


In [78]:
users.describe()

,user_age
count,10000.000000
mean,27.646900
std,8.311686
min,16.000000
25%,21.000000
50%,26.000000
75%,32.000000
max,65.000000


## Master Table Creation  

Merge **ad_events**, **ads**, **campaigns**, and **users** into one **master_df** for analysis.  
This guarantees that each event includes ad metadata, campaign budget, and user demographic details


In [79]:
def create_master_table(ad_events, ads, campaigns, users):
    merged = pd.merge(ad_events, ads, on='ad_id', how='left')
    merged = pd.merge(merged, campaigns, on='campaign_id', how='left')
    merged = pd.merge(merged, users, on='user_id', how='left')
    return merged

master_df = create_master_table(ad_events, ads, campaigns, users)

# Inspect the final master table
print(master_df.head())
print(master_df.shape)

   event_id  ad_id user_id           timestamp day_of_week time_of_day  \
0         1    197   2359b 2025-07-26 00:19:56    saturday       night   
1         2     51   f9c67 2025-06-15 08:28:07      sunday     morning   
2         3     46   5b868 2025-06-27 00:40:02      friday       night   
3         4    166   3d440 2025-06-05 19:20:45    thursday     evening   
4         5     52   68f1a 2025-07-22 08:30:29     tuesday     morning   

   event_type  campaign_id ad_platform   ad_type  ... start_date   end_date  \
0        like            9    facebook   stories  ... 2025-05-25 2025-07-13   
1       share           26   instagram  carousel  ... 2025-04-01 2025-06-17   
2  impression           10   instagram  carousel  ... 2025-05-17 2025-07-21   
3  impression           14   instagram     image  ... 2025-04-15 2025-06-04   
4  impression            2   instagram   stories  ... 2025-04-16 2025-07-07   

  duration_days total_budget user_gender user_age  age_group         country  \


## Ad-Level Performance Metrics  

Calculate **impressions, clicks, purchases** for each ad and get **CTR** and **CVR**.

This answers: *Which ads are most engaging? Which ads actually convert?*  


In [80]:
# Event counts per AD: grouping the master table by ad-level identifiers, 
# so instead of having event_type as a row index, each event type becomes a separate column.
ad_summary = master_df.groupby(['ad_id', 'campaign_id', 'ad_platform', 'ad_type']).event_type.value_counts().unstack(fill_value=0).reset_index()

# Lowercase all column names
ad_summary.columns = [col.lower() for col in ad_summary.columns]

# Drop duplicate columns if any
ad_summary = ad_summary.loc[:, ~ad_summary.columns.duplicated()]

# If a column is missing, this adds it and fills it with 0, 
# even if one ad never got a click/purchase/impression
for col in ['impression', 'click', 'purchase']:
    if col not in ad_summary.columns:
        ad_summary[col] = 0


# CTR & CVR
ad_summary['CTR'] = ad_summary['click'] / ad_summary['impression']
ad_summary['CVR'] = ad_summary['purchase'] / ad_summary['click']

# Sort by CVR to see best performing ads
ad_summary = ad_summary.sort_values(by='CVR', ascending=False)
print(ad_summary.head(10))


     ad_id  campaign_id ad_platform  ad_type  click  comment  impression  \
188    189           41    facebook    image    184       18        1774   
55      56            1   instagram  stories    190       13        1735   
189    190           33    facebook    image    178       22        1706   
153    154           27   instagram  stories    194       22        1691   
178    179            5   instagram  stories    181       27        1699   
12      13            6    facebook  stories    191       21        1726   
179    180           47   instagram    image    215       25        1739   
93      94           14    facebook    video    184       13        1654   
121    122           10    facebook  stories    213       15        1715   
30      31           21    facebook    image    188       17        1691   

     like  purchase  share       CTR       CVR  
188    55        19      8  0.103720  0.103261  
55     74        19      8  0.109510  0.100000  
189    58       

**Interpretation**:  
- **High CTR + High CVR** → Top performing ads (both attractive & persuasive).  
- **High CTR + Low CVR** → Ads grab attention but don’t convert to purchases.  
- **Low CTR + High CVR** → Ads attract fewer clicks but those who click are high-intent.  


In [81]:
# sanity check

ad_summary.info()
ad_summary.head()

print(ad_summary[['impression','click','purchase']].sum())

ad_summary[['impression','click','purchase']].describe()

print('duplicated = ')
ad_summary['ad_id'].duplicated().sum()


<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 188 to 79
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ad_id        200 non-null    int64  
 1   campaign_id  200 non-null    int64  
 2   ad_platform  200 non-null    object 
 3   ad_type      200 non-null    object 
 4   click        200 non-null    int64  
 5   comment      200 non-null    int64  
 6   impression   200 non-null    int64  
 7   like         200 non-null    int64  
 8   purchase     200 non-null    int64  
 9   share        200 non-null    int64  
 10  CTR          200 non-null    float64
 11  CVR          200 non-null    float64
dtypes: float64(2), int64(8), object(2)
memory usage: 20.3+ KB
impression    343157
click          40495
purchase        2050
dtype: int64
duplicated = 


np.int64(0)

## Funnel Metrics  

Evaluate the ad funnel efficiency:  
- **Impression → Click** (CTR)  
- **Click → Purchase** (CVR)  
- **Overall Conversion** (from impression to purchase)  


In [93]:
# Ad Funnel Metrics

ad_summary['impression_to_click'] = ad_summary['click'] / ad_summary['impression']
ad_summary['click_to_purchase'] = ad_summary['purchase'] / ad_summary['click']
ad_summary['overall_conversion'] = ad_summary['purchase'] / ad_summary['impression']

print(ad_summary[['ad_id', 'campaign_id', 'ad_platform', 'ad_type',
                  'impression', 'click', 'purchase',
                  'impression_to_click', 'click_to_purchase', 'overall_conversion']].head(10))


     ad_id  campaign_id ad_platform  ad_type  impression  click  purchase  \
188    189           41    facebook    image        1774    184        19   
55      56            1   instagram  stories        1735    190        19   
189    190           33    facebook    image        1706    178        17   
153    154           27   instagram  stories        1691    194        18   
178    179            5   instagram  stories        1699    181        16   
12      13            6    facebook  stories        1726    191        16   
179    180           47   instagram    image        1739    215        18   
93      94           14    facebook    video        1654    184        15   
121    122           10    facebook  stories        1715    213        17   
30      31           21    facebook    image        1691    188        15   

     impression_to_click  click_to_purchase  overall_conversion  
188             0.103720           0.103261            0.010710  
55              0.10

**Interpretation**:  
- CTR = ability to capture attention.  
- CVR = ability to persuade once clicked.  
- Overall Conversion = best proxy for ROI at the ad level.  


In [83]:
# Campaign funnel metrics

campaign_funnel = master_df.groupby('campaign_id').event_type.value_counts().unstack(fill_value=0)

for col in ['impression', 'click', 'purchase']:
    if col not in campaign_funnel.columns:
        campaign_funnel[col] = 0

campaign_funnel['impression_to_click'] = campaign_funnel['click'] / campaign_funnel['impression']
campaign_funnel['click_to_purchase'] = campaign_funnel['purchase'] / campaign_funnel['click']
campaign_funnel['overall_conversion'] = campaign_funnel['purchase'] / campaign_funnel['impression']

print(campaign_funnel.head(10))


event_type   click  comment  impression  like  purchase  share  \
campaign_id                                                      
1              583       51        5089   189        35     25   
2              601       66        5308   175        31     26   
3              790       80        6775   243        43     41   
4             1194      121       10365   359        49     74   
5              588       70        5160   219        34     28   
6              577       59        5165   205        33     26   
7              607       65        5048   185        33     33   
8             1006      105        8563   310        47     55   
9             1237      123       10461   348        63     42   
10             773       80        6723   239        51     39   

event_type   impression_to_click  click_to_purchase  overall_conversion  
campaign_id                                                              
1                       0.114561           0.060034        

In [84]:
# Platfrom funnel metric

platform_funnel = master_df.groupby('ad_platform').event_type.value_counts().unstack(fill_value=0)

for col in ['impression', 'click', 'purchase']:
    if col not in platform_funnel.columns:
        platform_funnel[col] = 0

platform_funnel['impression_to_click'] = platform_funnel['click'] / platform_funnel['impression']
platform_funnel['click_to_purchase'] = platform_funnel['purchase'] / platform_funnel['click']
platform_funnel['overall_conversion'] = platform_funnel['purchase'] / platform_funnel['impression']

print(platform_funnel)


event_type   click  comment  impression  like  purchase  share  \
ad_platform                                                      
facebook     25641     2655      218042  7589      1333   1288   
instagram    14854     1487      125115  4556       717    690   

event_type   impression_to_click  click_to_purchase  overall_conversion  
ad_platform                                                              
facebook                0.117597           0.051987            0.006114  
instagram               0.118723           0.048270            0.005731  


## Segment by ad characteristics (platform, ad_type) (what & why)

What: group by ad_platform and ad_type to compare performance. <br>
Why: tells you what kind of creative and where work best. <br>
Which platform (Instagram vs Facebook, etc.) drives more engagement and conversions?
Which ad format (video vs image, etc.) converts better?
Which campaigns are most efficient in turning budget into purchases? <br>

Questions answered: Do videos convert better than images? Is Instagram or Facebook more efficient?

In [85]:
# platform summary
platform_summary = ad_summary.groupby('ad_platform')[['impression','click','purchase']].sum().reset_index()
platform_summary['ctr'] = platform_summary['click'] / platform_summary['impression'].replace(0, pd.NA)
platform_summary['cvr'] = platform_summary['purchase'] / platform_summary['click'].replace(0, pd.NA)
platform_summary

# ad type summary
adtype_summary = ad_summary.groupby('ad_type')[['impression','click','purchase']].sum().reset_index()
adtype_summary['ctr'] = adtype_summary['click'] / adtype_summary['impression'].replace(0, pd.NA)
adtype_summary['cvr'] = adtype_summary['purchase'] / adtype_summary['click'].replace(0, pd.NA)
adtype_summary.sort_values('cvr', ascending=False)


,ad_type,impression,click,purchase,ctr,cvr
2,stories,109987,12916,687,0.117432,0.053190
0,carousel,87545,10248,525,0.117060,0.051230
3,video,56603,6736,344,0.119004,0.051069
1,image,89022,10595,494,0.119016,0.046626


In [86]:
# aggregate ad_summary up to campaign
campaign_summary = (
    ad_summary
    .groupby('campaign_id')[['impression','click','purchase']]
    .sum()
    .reset_index()
)

# compute metrics safely
campaign_summary['ctr'] = campaign_summary['click'] / campaign_summary['impression'].replace(0, pd.NA)
campaign_summary['cvr'] = campaign_summary['purchase'] / campaign_summary['click'].replace(0, pd.NA)

# attach campaign metadata (budget, duration) from campaigns DF
campaign_summary = campaign_summary.merge(campaigns[['campaign_id','total_budget','duration_days','name']], on='campaign_id', how='left')

# ROI proxy: purchases per unit budget (handle zero budget)
campaign_summary['purchases_per_1000_budget'] = campaign_summary.apply(
    lambda r: (r['purchase'] / r['total_budget'] * 1000) if r['total_budget'] and r['total_budget']>0 else pd.NA,
    axis=1
)

# sort to inspect
campaign_summary.sort_values('cvr', ascending=False).head(10)


,campaign_id,impression,click,purchase,ctr,cvr,total_budget,duration_days,name,purchases_per_1000_budget
25,27,3355,397,33,0.118331,0.083123,12986.30,48,campaign_27_q3,2.541140
29,31,5093,598,42,0.117416,0.070234,72208.34,56,campaign_31_summer,0.581650
21,23,5066,611,42,0.120608,0.068740,26001.67,84,campaign_23_winter,1.615281
9,10,6723,773,51,0.114978,0.065977,19669.27,65,campaign_10_winter,2.592877
24,26,5178,569,37,0.109888,0.065026,44538.87,77,campaign_26_winter,0.830735
31,33,12155,1415,86,0.116413,0.060777,59264.68,87,campaign_33_summer,1.451117
36,38,13550,1610,97,0.118819,0.060248,71038.28,36,campaign_38_q3,1.365461
0,1,5089,583,35,0.114561,0.060034,24021.32,59,campaign_1_launch,1.457039
19,21,6797,819,49,0.120494,0.059829,37290.81,64,campaign_21_winter,1.313997
10,11,5146,625,37,0.121454,0.059200,79067.91,69,campaign_11_q3,0.467952


## Audience Segmentation  

Break down performance by user's **gender**, **age group**, and **country**.  

**Why?** To see which demographics drive conversions and where to allocate targeting budget.  


In [87]:
# by gender
gender_summary = (
    master_df
    .groupby(['user_gender','event_type'])
    .size()
    .unstack(fill_value=0)   # pivot event_type into columns
    .reset_index()
)

for col in ['impression','click','purchase']:
    if col not in gender_summary.columns:
        gender_summary[col] = 0

gender_summary['ctr'] = gender_summary['click'] / gender_summary['impression'].replace(0, pd.NA)
gender_summary['cvr'] = gender_summary['purchase'] / gender_summary['click'].replace(0, pd.NA)


gender_summary

event_type,user_gender,click,comment,impression,like,purchase,share,ctr,cvr
0,female,13997,1379,117722,4061,719,676,0.118899,0.051368
1,male,22326,2309,190329,6821,1123,1085,0.117302,0.050300
2,other,4172,454,35106,1263,208,217,0.118840,0.049856


In [88]:
# by age
age_summary = master_df.groupby(['age_group', 'event_type']).size().unstack(fill_value=0).reset_index()

for col in ['impression', 'click', 'purchase']:
    if col not in age_summary.columns:
        age_summary[col] = 0

age_summary['ctr'] = age_summary['click'] / age_summary['impression'].replace(0, pd.NA)
age_summary['cvr'] = age_summary['purchase'] / age_summary['click'].replace(0, pd.NA)

# print(age_summary.sort_values('cvr', ascending=False))

age_summary

event_type,age_group,click,comment,impression,like,purchase,share,ctr,cvr
0,16-17,3536,373,30156,1051,159,165,0.117257,0.044966
1,18-24,12532,1257,106744,3766,644,631,0.117402,0.051388
2,25-34,16855,1727,142248,4987,857,806,0.118490,0.050845
3,35-44,5885,630,50020,1798,307,305,0.117653,0.052167
4,45-54,1348,128,11134,424,66,58,0.121071,0.048961
5,55-65,339,27,2855,119,17,13,0.118739,0.050147


In [95]:
# by location
country_summary = master_df.groupby(['country', 'event_type']).size().unstack(fill_value=0).reset_index()

for col in ['impression', 'click', 'purchase']:
    if col not in country_summary.columns:
        country_summary[col] = 0

country_summary['ctr'] = country_summary['click'] / country_summary['impression'].replace(0, pd.NA)
country_summary['cvr'] = country_summary['purchase'] / country_summary['click'].replace(0, pd.NA)


print('LOCATION')
print(country_summary.sort_values('cvr', ascending=False).head(10))  
# country_summary


LOCATION
event_type         country  click  comment  impression  like  purchase  share  \
6                    japan   1961      215       16812   568       127     97   
7                   mexico   2059      212       17803   620       127    104   
9            united states  12292     1222      103560  3665       656    607   
2                   canada   4086      386       34488  1280       201    196   
5                    india   3873      393       32125  1164       189    169   
1                   brazil   2439      241       20909   769       119    130   
8           united kingdom   6027      618       51761  1784       290    295   
4                  germany   3310      374       28167   957       152    170   
0                australia   2946      316       24476   856       127    132   
3                   france   1502      165       13056   482        62     78   

event_type       ctr       cvr  
6           0.116643  0.064763  
7           0.115655  0.061680  


**Interpretation**:  
- Certain genders may show higher CTR but lower CVR (interest vs intent gap).  
- Younger users often have higher engagement but weaker purchasing power.  
- Geographic differences highlight strong and weak markets.  


## Campaign-Level ROI & Budget Efficiency  

Roll up metrics at the **campaign level** to examine:  
- CTR / CVR across campaigns  
- Purchases relative to budget (efficiency)  
- Cost per Purchase (CPP)  
- Purchases per Day (time efficiency)  


In [90]:
# Merge with Budget & Duration

campaign_summary = (
    ad_summary
    .groupby('campaign_id')[['impression','click','purchase']]
    .sum()
    .reset_index()
)

# CTR and CVR
campaign_summary['ctr'] = campaign_summary['click'] / campaign_summary['impression'].replace(0, pd.NA)
campaign_summary['cvr'] = campaign_summary['purchase'] / campaign_summary['click'].replace(0, pd.NA)

# merge metadata
campaign_summary = campaign_summary.merge(
    campaigns[['campaign_id','name','total_budget','duration_days']],
    on='campaign_id',
    how='left'
)


In [91]:
# purchases per $1000 budget
campaign_summary['purchases_per_1000_budget'] = campaign_summary.apply(
    lambda r: (r['purchase'] / r['total_budget'] * 1000)
              if r['total_budget'] and r['total_budget']>0 else pd.NA,
    axis=1
)

# cost per purchase (CPP)
campaign_summary['cost_per_purchase'] = campaign_summary.apply(
    lambda r: (r['total_budget'] / r['purchase'])
              if r['purchase'] and r['purchase']>0 else pd.NA,
    axis=1
)

# purchases per day
campaign_summary['purchases_per_day'] = campaign_summary.apply(
    lambda r: (r['purchase'] / r['duration_days'])
              if r['duration_days'] and r['duration_days']>0 else pd.NA,
    axis=1
)


In [92]:
# rank campaigns 

# by conversion efficiency
top_campaigns = campaign_summary.sort_values('purchases_per_1000_budget', ascending=False).head(10)

top_campaigns[['campaign_id','name','total_budget','duration_days',
               'impression','click','purchase',
               'ctr','cvr','purchases_per_1000_budget','cost_per_purchase','purchases_per_day']]


,campaign_id,name,total_budget,duration_days,impression,click,purchase,ctr,cvr,purchases_per_1000_budget,cost_per_purchase,purchases_per_day
40,42,campaign_42_summer,7918.04,47,13814,1621,67,0.117345,0.041333,8.461690,118.179701,1.425532
27,29,campaign_29_winter,19773.66,82,11806,1438,60,0.121802,0.041725,3.034340,329.561000,0.731707
2,3,campaign_3_winter,14343.25,56,6775,790,43,0.116605,0.054430,2.997926,333.563953,0.767857
12,13,campaign_13_winter,21855.42,73,10424,1226,65,0.117613,0.053018,2.974091,336.237231,0.890411
32,34,campaign_34_winter,26104.30,60,10306,1251,69,0.121386,0.055156,2.643243,378.323188,1.150000
9,10,campaign_10_winter,19669.27,65,6723,773,51,0.114978,0.065977,2.592877,385.671961,0.784615
25,27,campaign_27_q3,12986.30,48,3355,397,33,0.118331,0.083123,2.541140,393.524242,0.687500
41,44,campaign_44_q3,15914.87,70,6837,802,39,0.117303,0.048628,2.450538,408.073590,0.557143
21,23,campaign_23_winter,26001.67,84,5066,611,42,0.120608,0.068740,1.615281,619.087381,0.500000
8,9,campaign_9_launch,40094.07,49,10461,1237,63,0.118249,0.050930,1.571305,636.413810,1.285714


**Interpretation**:  
- **High purchases per $1000 budget** = strong ROI campaigns.  
- **Low cost per purchase** = efficient spend.  
- **High purchases/day** = quick impact campaigns.  
